# LAB 01: LAYOFFS 2023

## Mô tả đồ án:
- Từ nguồn dữ liệu lấy được từ Kaggle về vấn đề sa thải nhân viên ở các công ty trên toàn thế giới, đồ án này nhằm mục đích phân tích dữ liệu đã có để có được cái nhìn khách quan hơn về vấn đề này.

---

### Import các thư viện

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

---

### <span style="color:yellow"> A. Thu thập dữ liệu</span>


#### Ngữ cảnh thực hiện tìm kiếm dữ liệu

#### Chủ đề

#### Có cần phải tiền xử lý dữ liệu hay không và nếu có thì nhóm sinh viên cần phải xử lý như thế nào?

#### Nguồn
Dữ liệu được lấy trên Kaggle dưới file csv, đường link dẫn đến  [Ở đây](https://www.kaggle.com/datasets/swaptr/layoffs-2022?resource=download)

#### Người ta có cho phép sử dụng dữ liệu như thế này hay không?

#### Người ta đã thu thập dữ liệu này như thế nào? Phương pháp thực hiện là gì?

#### Đọc dữ liệu

In [9]:
df=pd.read_csv('./datasets/layoffs.csv')
df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,Atlassian,Sydney,Other,500.0,0.05,2023-03-06,Post-IPO,Australia,210.0
1,SiriusXM,New York City,Media,475.0,0.08,2023-03-06,Post-IPO,United States,525.0
2,Alerzo,Ibadan,Retail,400.0,NaN,2023-03-06,Series B,Nigeria,16.0
3,UpGrad,Mumbai,Education,120.0,NaN,2023-03-06,Unknown,India,631.0
4,Loft,Sao Paulo,Real Estate,340.0,0.15,2023-03-03,Unknown,Brazil,788.0


---

### <span style="color:yellow"> B. Khám phá dữ liệu</span>


#### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

#### Mỗi cột có ý nghĩa gì?

#### Mỗi cột hiện đang có kiểu dữ liệu gì?

#### Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

#### Có cần phải tiền xử lý dữ liệu hay không và nếu có thì nhóm sinh viên cần phải xử lý như thế nào?

In [10]:
# nếu cần tiền xử lí